In [125]:
import pandas as pd
import numpy as np
import jieba
import jieba.analyse
import codecs
import requests
import json
import time
import csv
import urllib
import random
from pandas.core.frame import DataFrame

In [126]:
colnames=['ID','Href','Blog','PubTime','Like','Comment','Transfer']

In [127]:
df1 = pd.read_csv('yanshuji.csv',names=colnames)
df1['PubTime']=pd.to_datetime(df1['PubTime'])
df1=df1[df1['PubTime']<'2018-05-25']

In [128]:
df2= pd.read_csv('yanchunfeng.csv',names=colnames)
df2['PubTime']=pd.to_datetime(df2['PubTime'])
df2=df2[df2['PubTime']<'2018-05-25']
df2=df2[df2['Blog'].str.contains('严书记')==False]

In [129]:
df3= pd.read_csv('yanfuren.csv',names=colnames)
df3['PubTime']=pd.to_datetime(df3['PubTime'])
df3=df3[df3['PubTime']<'2018-05-25']
df3=df3[df3['Blog'].str.contains('严书记')==False]
df3=df3[df3['Blog'].str.contains('严春风')==False]

In [130]:
df4= pd.read_csv('youeryuan.csv',names=colnames)
df4['PubTime']=pd.to_datetime(df4['PubTime'])
df4=df4[df4['PubTime']<'2018-05-25']
df4=df4[df4['Blog'].str.contains('严书记')==False]
df4=df4[df4['Blog'].str.contains('严春风')==False]
df4=df4[df4['Blog'].str.contains('严夫人')==False]

In [131]:
df_all=pd.concat([df1,df2,df3,df4])

In [132]:
df_all=df_all[df_all['Blog'].str.contains('优惠券')==False]

In [133]:
df_all.to_excel('total_data.xlsx')

In [134]:
jieba.analyse.set_stop_words('stop.txt')

In [135]:
def tosentence(df):
    w1=0.2
    w2=0.4
    w3=0.4
    sentence=''
    for row in df.iterrows():
        sentence+=row[1]['Blog']*int(w1*row[1]['Like']+w2*row[1]['Comment']+w3*row[1]['Transfer'])
    return sentence  

In [139]:
#dfbytime=[]
file=open('senti3.txt','w')
file.write(','.join(['Time Period','Total','Negtive','Middle','Positive','Weighted Sentiment'])+'\n')
for date in range(11,25):
    for hour in range(0,24,4):
        print(date,hour)
        pubtime='2018-05-%d %d:00'%(date,hour%24)
        temp1=df_all[df_all['PubTime']>pd.to_datetime(pubtime)]
        pubtime='2018-05-%d %d:00'%(date,(hour+4)%24)
        if not (hour+4)%24:
            pubtime='2018-05-%d 23:59'%(date)
        temp2=temp1[temp1['PubTime']<pd.to_datetime(pubtime)]
        #dfbytime.append(temp2)
        #temp2.to_excel('data_by_4hours/05-%d %d~%d.xlsx'%(date,hour,hour+4))
        #sentence=tosentence(temp2)
        #keywords = jieba.analyse.extract_tags(sentence, topK=20, withWeight=True)
        #f=open('weighted_keywords/05-%d %d~%d.txt'%(date,hour,hour+4),'w')
        #for item in keywords:
            #f.write(item[0]+' '+str(item[1])+'\n')
        #f.close()
        #sentence2=' '.join(temp2['Blog'])
        #keywords = jieba.analyse.extract_tags(sentence2, topK=20, withWeight=True)
        #f=open('origin_keywords/05-%d %d~%d.txt'%(date,hour,hour+4),'w')
        #for item in keywords:
            #f.write(item[0]+' '+str(item[1])+'\n')
        #f.close()
        output=dfsummary(temp2,date,hour)
        file.write(output)
        file.flush()
        file.write('\n')
file.close()

15 20


In [136]:
def analysis(string):
    access_token = '24.04828717b0a4530749f1f9d33b776cf9.2592000.1569760669.282335-16947555'
    url_origin = 'https://aip.baidubce.com/rpc/2.0/nlp/v1/sentiment_classify?charset=UTF-8&'
    url = "https://aip.baidubce.com/rpc/2.0/nlp/v1/sentiment_classify?access_token="+access_token  # API
    headers = {'content-type': 'application/json',}
    text = {"text": string}
    analyze_response = requests.post(url = url, headers=headers, data = json.dumps(text).encode('utf-8'))
    analyze_response.encoding = 'gbk'
    j_analyze = json.loads(analyze_response.text)
    return(j_analyze['items'][0]['confidence'],j_analyze['items'][0]['sentiment'])

In [137]:
def dfsummary(df,date=1,hour=1):
    w1=0.2
    w2=0.4
    w3=0.4
    sumlist=[]
    conflist=[]
    weighted_senti=0
    for row in df.iterrows():
        try:
            conf,senti= analysis(row[1]['Blog'])
        except:
            print('error')
            #continue
        sumlist.append(senti)
        conflist.append(conf)
        weighted_senti+=(senti-1)*(w1*row[1]['Like']+w2*row[1]['Comment']+w3*row[1]['Transfer'])*conf
        time.sleep(random.uniform(0.5,1.0))
    l=[sumlist,conflist]
    l=pd.DataFrame(data=l)
    l.to_csv('data_by_4hours/senti/05-%d %d~%d.csv'%(date,hour,hour+4))
    return(','.join(['05-%d %d~%d'%(date,hour,hour+4),str(len(sumlist)),
                         str(sumlist.count(0)),str(sumlist.count(1)),str(sumlist.count(2)),str(weighted_senti)]))

In [110]:
def get_access_token():
    """
    获取百度AI平台的Access Token
    """
    # 使用你的API Key及Secret Key
    host = 'https://aip.baidubce.com/oauth/2.0/token?grant_type=client_credentials&client_id=FIGcoqTMBM95h0GlZQneRz8U&client_secret=qQs3qaPmSIlEkIPw3fIcyf0EGp2rfTXD'
    request = urllib.request.Request(host)
    request.add_header('Content-Type', 'application/json; charset=UTF-8')
    response = urllib.request.urlopen(request)
    content = response.read().decode('utf-8')
    rdata = json.loads(content)
    return rdata['access_token']

In [111]:
get_access_token()

'24.04828717b0a4530749f1f9d33b776cf9.2592000.1569760669.282335-16947555'